In [3]:
import json
import os
import numpy as np
# from osgeo import gdal, gdalconst, osr
from pathlib import Path
import time
import rasterio
import requests
from requests.auth import HTTPBasicAuth
import shutil
import zipfile

REQUEST_ID = 0
ORDER_ID = None

In [4]:
ORDER_ID = 'c3d06c17-bb29-49dd-be3a-a2616b2ecb26'

In [5]:
NAME = "Planet Downloader"
NB_USER = os.getenv('NB_USER')
BASE = os.getcwd()
DATA_DIR = Path(f"/home/{NB_USER}/work")

API_KEY_DIR = DATA_DIR / ".secret/planet_api_key.json"
LOAD_DIR = DATA_DIR / "satellite_imagery"
RESULTS_DIR = DATA_DIR / "results/planet_downloader"
PBD_DIR = DATA_DIR / "notebooks/planet_downloader"

# API_KEY = os.path.join(DATA_DIR, ".secret/planet_api_key.json")
# LOAD_DIR = os.path.join(DATA_DIR, "satellite_imagery")
# RESULTS_DIR = os.path.join(DATA_DIR, "results/planet_downloader")
# PBD_DIR = os.path.join(DATA_DIR, "notebooks/planet_downloader")

RESULTS_DIR.mkdir(parents=True, exist_ok=True)
# os.makedirs(RESULTS_DIR, exist_ok=True)

In [6]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

In [7]:
# API Key stored in a planet_api_key.json file
with open(API_KEY_DIR, 'r') as f:
    PLANET_API_KEY = json.load(f)['api_key']

In [8]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')

In [9]:
def get_order_url():
    return f'{orders_url}/{ORDER_ID}'

In [10]:
def poll_for_success(order_url, auth):
    count = 0
    while True:
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)

In [11]:
def download_results(response, overwrite=False):
    load_path = LOAD_DIR / response['name']
    load_path.mkdir(parents=True, exist_ok=True)
    
    results_urls = [r['location'] for r in response['_links']['results']]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = load_path / name
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [12]:
def get_ziped_files_from_manifest(response):
    load_path = LOAD_DIR / response['name'] / ORDER_ID
    with open(load_path / 'manifest.json') as f:
        files = json.load(f)['files']
    ziped_files = []    
    for file in files:
        if file['media_type'] == 'application/zip':
            file_path = load_path / file['path']
            ziped_files.append(file_path)
    return ziped_files

In [14]:
def unzip_results(ziped_files):
    for zip_file in ziped_files:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(path=LOAD_DIR / f'response["name"]')

In [15]:
def get_tif_files_list_from_manifest(response):
    load_path = LOAD_DIR / response['name']
    with open(load_path / 'manifest.json') as f:
        files = json.load(f)['files']
    tif_files = []
    for file in files:
        if file['path'].endswith('.tif'):
            tif_files.append(load_path / file['path'])
    return tif_files

In [16]:
def copy_files_to_results(response, tif_files_list):
    (RESULTS_DIR / str(REQUEST_ID)).mkdir(parents=True, exist_ok=True)
    for file in tif_files_list:
        dst_file = RESULTS_DIR / str(REQUEST_ID) / file.name
        shutil.copy(file, dst_file)
        print(f'{file.name} was copied to {dst_file}')
    print('all images were copied')

In [17]:
order_url = get_order_url()

In [18]:
order_url

'https://api.planet.com/compute/ops/orders/v2/c3d06c17-bb29-49dd-be3a-a2616b2ecb26'

In [19]:
poll_for_success(order_url, auth)

success


In [20]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']

In [21]:
response

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/c3d06c17-bb29-49dd-be3a-a2616b2ecb26',
  'results': [{'delivery': 'success',
    'expires_at': '2022-04-11T09:26:26.487Z',
    'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2NDk2NjkxODYsInN1YiI6InpwM0RzRDRkWXpOVWE4Y0NxaFJpdmZNeW4rcGZnTkNaNG5NV0RXMk81dmhKTUFzZHUxRzYycEhBdmg3eW15L3BaWkw5aGUxU3hIVEtITGJ6SXRjUWtnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9jM2QwNmMxNy1iYjI5LTQ5ZGQtYmUzYS1hMjYxNmIyZWNiMjYvbWFuaWZlc3QuanNvbj9FeHBpcmVzPTE2NDk2NjkxODZcdTAwMjZHb29nbGVBY2Nlc3NJZD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tXHUwMDI2U2lnbmF0dXJlPUJFNVJSR0dVaTFYcmd1QTlkZno0TlhVbnR5QVhsRzdzaWg4NHBMSXVpdmtzdW16TGpYcTJmQ3luMWNCWkJ3eVNIR2pTTXJxV2ZTM1BiZWx5ejR1e

In [55]:
order_name = response['name']

In [56]:
order_name

'Mariupol_21-03-2022_SS'

In [57]:
results

[{'delivery': 'success',
  'expires_at': '2022-04-08T18:12:46.019Z',
  'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2NDk0NDE1NjYsInN1YiI6IjZjSHFFQVpFZHNDWTZ5UVoweld3b1BmaEZ4OGRlWlJxMnhkOWFpWExHRnE3MjhaVjhhU2tZTWkxdGwxQ3B6TEpyS3Q0MTNkRWQrVGpOT3htYnRNU1ZBPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9jM2QwNmMxNy1iYjI5LTQ5ZGQtYmUzYS1hMjYxNmIyZWNiMjYvbWFuaWZlc3QuanNvbj9FeHBpcmVzPTE2NDk0NDE1NjZcdTAwMjZHb29nbGVBY2Nlc3NJZD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tXHUwMDI2U2lnbmF0dXJlPWpEeTZVOEE5SVRoOXpQVlJjVzlzdUFKNjFVR1BKdEZuOTN5cU4zMjQ1dGRSRGdoZzdMTTFDdkp3bUtQNERNZVFIZ1Vkd1RXNnZURVRCT2tHUVVzN2IlMkJPZ21nZE9ZTSUyRld0ZkJ6V084SHV6SW51WHJiSzh4dndzMVFDJTJGaURFeFF6JTJCUldXd01TdXc1TlV4aUhnSzlhMEkzM1J2TDhzT2g1UHd6cmQ2MiUy

In [58]:
download_results(response)

2 items to download
downloading c3d06c17-bb29-49dd-be3a-a2616b2ecb26/manifest.json to /home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/c3d06c17-bb29-49dd-be3a-a2616b2ecb26/manifest.json
downloading c3d06c17-bb29-49dd-be3a-a2616b2ecb26/Mariupol_21-03-2022_SS_skysatcollect_analytic_sr_udm2.zip to /home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/c3d06c17-bb29-49dd-be3a-a2616b2ecb26/Mariupol_21-03-2022_SS_skysatcollect_analytic_sr_udm2.zip


In [76]:
ziped_files_list = get_ziped_files_from_manifest(response)

In [77]:
ziped_files_list

[PosixPath('/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/c3d06c17-bb29-49dd-be3a-a2616b2ecb26/Mariupol_21-03-2022_SS_skysatcollect_analytic_sr_udm2.zip')]

In [79]:
unzip_results(ziped_files_list)

In [89]:
tif_files_list = get_tif_files_list_from_manifest(response)

In [90]:
tif_files_list

[PosixPath('/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/files/SkySatCollect/20220321_112949_ssc11_u0005/analytic_sr_udm2/20220321_112949_ssc11_u0005_analytic_SR.tif'),
 PosixPath('/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/files/SkySatCollect/20220321_112949_ssc11_u0005/analytic_sr_udm2/20220321_112949_ssc11_u0005_analytic_udm.tif'),
 PosixPath('/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/files/SkySatCollect/20220321_112949_ssc11_u0005/analytic_sr_udm2/20220321_112949_ssc11_u0005_udm2.tif'),
 PosixPath('/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/files/SkySatCollect/20220321_112949_ssc11_u0004/analytic_sr_udm2/20220321_112949_ssc11_u0004_analytic_SR.tif'),
 PosixPath('/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/files/SkySatCollect/20220321_112949_ssc11_u0004/analytic_sr_udm2/20220321_112949_ssc11_u0004_analytic_udm.tif'),
 PosixPath('/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/files/SkySatCollect

In [102]:
copy_files_to_results(response, tif_files_list)

20220321_112949_ssc11_u0005_analytic_SR.tif was copied to /home/jovyan/work/results/planet_downloader/0/20220321_112949_ssc11_u0005_analytic_SR.tif
20220321_112949_ssc11_u0005_analytic_udm.tif was copied to /home/jovyan/work/results/planet_downloader/0/20220321_112949_ssc11_u0005_analytic_udm.tif
20220321_112949_ssc11_u0005_udm2.tif was copied to /home/jovyan/work/results/planet_downloader/0/20220321_112949_ssc11_u0005_udm2.tif
20220321_112949_ssc11_u0004_analytic_SR.tif was copied to /home/jovyan/work/results/planet_downloader/0/20220321_112949_ssc11_u0004_analytic_SR.tif
20220321_112949_ssc11_u0004_analytic_udm.tif was copied to /home/jovyan/work/results/planet_downloader/0/20220321_112949_ssc11_u0004_analytic_udm.tif
20220321_112949_ssc11_u0004_udm2.tif was copied to /home/jovyan/work/results/planet_downloader/0/20220321_112949_ssc11_u0004_udm2.tif


In [4]:
raster_path = '/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/files/SkySatCollect/20220321_112949_ssc11_u0005/analytic_sr_udm2/20220321_112949_ssc11_u0005_analytic_SR.tif'
raster_dst = '/home/jovyan/work/satellite_imagery/Mariupol_21-03-2022_SS/files/SkySatCollect/20220321_112949_ssc11_u0005/rgb.tif'

In [13]:
with rasterio.open(raster_path, "r") as src:
#     bands, rows, columns =  array.shape
#     for i in range(0, bands):
#         color = src.colorinterp[i]
#         print(color)
#     array = src.read()
#     print(array.shape)
    profile = src.profile
    profile['photometric'] = "RGB"
    print(profile)
    with rasterio.open(raster_dst, 'w', **profile) as dst:
        dst.write(src.read())
#     meta = src.meta
#     print(meta)

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': 0.0, 'width': 13021, 'height': 13751, 'count': 4, 'crs': CRS.from_epsg(32637), 'transform': Affine(0.5, 0.0, 387392.5,
       0.0, -0.5, 5216186.5), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel', 'photometric': 'RGB'}


(4, 13751, 13021)

In [ ]:
def image_normalization(raster_path):
    with rasterio.open(raster_path, "r") as src:
        array = src.read()
        print(array.shape)

In [14]:
dataset = gdal.Open(raster_path)
proj = osr.SpatialReference(wkt=dataset.GetProjection())
print(proj)
epsg = proj.GetAttrValue("AUTHORITY", 1)
if epsg != "3857":
    gdal.Warp(raster_dst,
              dataset,
              resampleAlg=gdalconst.GRIORA_Cubic,
              outputType=gdal.GDT_Byte,
              dstSRS="EPSG:3857",
              dstNodata=0
              )

NameError: name 'gdal' is not defined

In [8]:
def transform_raster(raster_path, new_raster_path, date=None, request_id=0):
    with rasterio.open(raster_path, "r") as src:
        updated_meta = src.profile.copy()
        updated_meta.update({"count": 3,
                             "bands": 3,
                             "nodata": 0,
                             "dtype": 'uint8'})
        with rasterio.open(new_raster_path, "w", **updated_meta) as dst:
            rgb = [3, 2, 1]
            dst.update_tags(start_date=date, end_date=date, request_id=request_id)
            for num, band in enumerate(range(1,4)):
                cur_band = src.read(rgb[num])
                max_ = np.percentile(cur_band, 98)
                cur_band[cur_band > max_] = max_
                cur_band = (cur_band / max_) * 255
                cur_band = cur_band.astype(np.uint8)
                dst.write(cur_band, band)

In [11]:
transform_raster(raster_path, raster_dst)